In [138]:
import os
import imp

import numpy as np
import pandas as pd
import xarray as xr

import climatools.io.cesm as climaio
import climatools.aerosol.modal_aero_sw as aerosw
import climatools.aerosol.aerosol_constants as aeroconst
import climatools.aerosol.aerosol as climaerosol

imp.reload(aerosw)
imp.reload(aeroconst)
imp.reload(climaerosol)

SyntaxError: invalid syntax (aerosol.py, line 27)

In [126]:
aerosw.modal_aero_sw?

### timeit: getting the constants

Data saved on the netCDF files in the relevant namelists are loaded by xarray when *climatools.aerosol.aerosol_constants* is imported.

These then have to be reshaped or combined into arrays that are compatible with the f2py3-compiled subroutine *modal_aero_sw()*.

In [86]:
%timeit aeroconst.get_specdens()

100 loops, best of 3: 6.08 ms per loop


In [85]:
%timeit aeroconst.get_specrefindex()

10 loops, best of 3: 35.2 ms per loop


In [101]:
%timeit aeroconst.get_extpsw()

1000 loops, best of 3: 162 µs per loop


In [105]:
%timeit aeroconst.get_abspsw()

1000 loops, best of 3: 162 µs per loop


In [109]:
%timeit aeroconst.get_asmpsw()

1000 loops, best of 3: 163 µs per loop


In [114]:
%timeit aeroconst.get_refrtabsw()

10000 loops, best of 3: 146 µs per loop


In [121]:
%timeit  aeroconst.get_refitabsw()

1000 loops, best of 3: 168 µs per loop


In [119]:
%timeit aeroconst.get_crefwsw()

100 loops, best of 3: 3.18 ms per loop


In [122]:
%%timeit

import climatools.aerosol.aerosol_constants as aeroconst

aeroconst.get_specdens()
aeroconst.get_specrefindex()
aeroconst.get_extpsw()
aeroconst.get_abspsw()
aeroconst.get_asmpsw()
aeroconst.get_refrtabsw()
aeroconst.get_refitabsw()
aeroconst.get_crefwsw()

10 loops, best of 3: 44.7 ms per loop


### timeit: dynamic data

In [131]:
# load a CAM history file from some previous global run
filedir = '/nuwa_cluster/home/jackyu/climatools/climatools/aerosol'
filepath = os.path.join(filedir, 'f19_g16.B1850.std.cam2.h0.0001-06.nc')
ds = climaio.load_camhistory(readfrom=filepath)

In [136]:
%timeit climaerosol.get_raer(ds)

1 loops, best of 3: 369 ms per loop


In [137]:
%timeit climaerosol.wateruptake(ds)

1 loops, best of 3: 2.01 s per loop


### timeit: with fake random data

In [8]:
np.random.random(8)

array([ 0.04399301,  0.08889587,  0.32317629,  0.2495431 ,  0.51951521,
        0.90977068,  0.66340565,  0.70196632])

In [25]:
def generate_fake_random_data(pcols=1):
    pver = 30
    ntot_amode = 3
    nspec_max = 6
    ncoef = 5
    prefr = 7
    prefi = 10
    nswbands = 14
    
    mass = np.random.random((pcols, pver))
    specmmr = np.random.random((nspec_max, ntot_amode, pcols, pver))
    dgnumwet = np.random.random((pcols, pver, ntot_amode))
    qaerwat = np.random.random((pcols, pver, ntot_amode))
    specdens = np.random.random((nspec_max, ntot_amode))
    specrefindex = np.random.random((nspec_max, ntot_amode, nswbands)) 
    extpsw = np.random.random((ncoef, prefr, prefi, ntot_amode, nswbands))
    abspsw = np.random.random((ncoef, prefr, prefi, ntot_amode, nswbands))
    asmpsw = np.random.random((ncoef, prefr, prefi, ntot_amode, nswbands))
    refrtabsw = np.random.random((prefr, nswbands))
    refitabsw = np.random.random((prefi, nswbands))
    crefwsw = np.random.random((nswbands,)) + 1j * np.random.random((nswbands,))
    
    return mass, specmmr, dgnumwet, qaerwat, specdens, specrefindex, \
           extpsw, abspsw, asmpsw, refrtabsw, refitabsw, crefwsw
    

  

In [27]:
%timeit aerosw.modal_aero_sw(*generate_fake_random_data(pcols=180*96))  

1 loops, best of 3: 7.67 s per loop


In [146]:
ds

<xarray.Dataset>
Dimensions:              (chars: 8, ilev: 31, lat: 96, lev: 30, lon: 144, mode: 3, nbnd: 2, slat: 95, slon: 144, species: 6, time: 1)
Coordinates:
  * lev                  (lev) float64 3.643 7.595 14.36 24.61 38.27 54.6 ...
  * ilev                 (ilev) float64 2.255 5.032 10.16 18.56 30.67 45.87 ...
  * time                 (time) float64 181.0
  * lat                  (lat) float64 -90.0 -88.11 -86.21 -84.32 -82.42 ...
  * lon                  (lon) float64 0.0 2.5 5.0 7.5 10.0 12.5 15.0 17.5 ...
  * slat                 (slat) float64 -89.05 -87.16 -85.26 -83.37 -81.47 ...
  * slon                 (slon) float64 -1.25 1.25 3.75 6.25 8.75 11.25 ...
  * chars                (chars) int64 0 1 2 3 4 5 6 7
  * nbnd                 (nbnd) int64 0 1
  * mode                 (mode) int64 1 2 3
  * species              (species) int64 1 2 3 4 5 6
Data variables:
    hyam                 (lev) float64 0.003643 0.007595 0.01436 0.02461 ...
    hybm                 (lev) flo